# Libraries

In [31]:
import numpy as np
import pandas as pd
import sympy as sym
pi = np.pi
import cvxpy as cp

# Translation operator 

## Ideal projectors

In [32]:
def proj(j,n):
#     mat = np.zeros((3,3))
    temp1 = np.cos(pi/n)/(1+np.cos(pi/n))
    temp2 = 1-temp1
    temp3 = j*pi*(n-1)/n
    vec = np.asarray([np.sqrt(temp1),np.sqrt(temp2)*np.sin(temp3), np.sqrt(temp2)*np.cos(temp3)])
    mat = np.outer(vec,vec)
    return mat    

In [33]:
# Orthogonality check
i = 0
np.dot(proj((i+1) % 5,5),proj(i,5))

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [34]:
#Projection check
i = 0
proj(i,5)@proj(i,5) - proj(i,5) <= 1.0e-10

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], dtype=bool)

## Translation operator

In [35]:
T = np.matrix([[0,0,1],
               [1,0,0],
               [0,1,0]])

In [36]:
np.transpose(T)@T

matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

## 5 cycle

In [37]:
P1 = proj(1,5)
P2 = proj(2,5)
P3 = proj(3,5)
P4 = proj(4,5)
P5 = proj(5,5)

In [38]:
P1

array([[ 0.4472136 ,  0.29225023, -0.40224793],
       [ 0.29225023,  0.19098301, -0.26286556],
       [-0.40224793, -0.26286556,  0.3618034 ]])

In [39]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
x3 = sym.Symbol('x3')
x4 = sym.Symbol('x4')
x5 = sym.Symbol('x5')
x6 = sym.Symbol('x6')
x7 = sym.Symbol('x7')
x8 = sym.Symbol('x8')
x9 = sym.Symbol('x9')
x10 = sym.Symbol('x10')

In [40]:
#LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)
 LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P1@P3) + x7*(P2@P4) + x8*(P3@P5) + x9*(P4@P1) +x10*(P5@P2)

In [41]:
LHSP = pd.DataFrame(LHS)

In [42]:
LHSP

,0,1,2
0,0.447213595499958*x1 + 0.276393202250021*x10 +...,0.292250229469488*x1 - 0.292250229469488*x10 -...,-0.402247932095355*x1 + 0.0949578557832232*x10...
1,0.292250229469488*x1 - 1.50528723346159e-16*x1...,0.190983005625053*x1 + 1.59164746388624e-16*x1...,-0.262865556059567*x1 - 5.17157610475791e-17*x...
2,-0.402247932095355*x1 + 0.307290076312132*x10 ...,-0.262865556059567*x1 - 0.324919696232906*x10 ...,0.361803398874989*x1 + 0.105572809000084*x10 +...


In [43]:
LHS;

In [44]:
Equations = LHS - T

In [45]:
Equations;

In [46]:
P1@P3 - P1@P4

array([[ -5.55111512e-17,   4.72870805e-01,   3.43560750e-01],
       [ -5.55111512e-17,   3.09016994e-01,   2.24513988e-01],
       [  5.55111512e-17,  -4.25325404e-01,  -3.09016994e-01]])

In [47]:
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solution5 = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10))

In [48]:
solution5

{x9: 1.1136394303114*x10 + 837921279079462.0,
 x8: -0.311732980855484*x10 + 517863830367898.0,
 x7: -1.30630100789441*x10 - 517863830367896.0,
 x4: 0.192661577583011*x10 - 320057448711568.0,
 x5: -0.688267019144514*x10 - 517863830367896.0,
 x6: -0.495605441561502*x10 - 837921279079464.0,
 x3: 0.807338422416987*x10 + 320057448711569.0,
 x2: 0.306301007894408*x10 + 517863830367895.0,
 x1: -0.618033988749893*x10 + 0.365319220073288}

### Sanity check

In [49]:
y1 = sym.Symbol('y1')
y2 = sym.Symbol('y2')
eqset = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset.append(LHS[i,j])
solution = sym.solve((eqset),(y1,y2))

In [50]:
solution

[]

### Isometry

In [51]:
# S = TUVU
# We need three basis vectors. Let these be P1, P2 and P3.
# First we construct the basis vectors
def basis(j,n):
    temp1 = np.cos(pi/n)/(1+np.cos(pi/n))
    temp2 = 1-temp1
    temp3 = j*pi*(n-1)/n
    vec = np.asarray([np.sqrt(temp1),np.sqrt(temp2)*np.sin(temp3), np.sqrt(temp2)*np.cos(temp3)])
    return vec      

In [52]:
b1 = basis(0,5)
b2 = basis(1,5)
b3 = np.cross(b1,b2)

In [53]:
# Sanity check
np.inner(b2,b3)

-2.7755575615628914e-17

In [54]:
coeff = [solution5[x1],solution5[x2],solution5[x3],solution5[x4],solution5[x5],solution5[x6],solution5[x7],solution5[x8],solution5[x9]]

In [55]:
coeff

[-0.618033988749893*x10 + 0.365319220073288,
 0.306301007894408*x10 + 517863830367895.0,
 0.807338422416987*x10 + 320057448711569.0,
 0.192661577583011*x10 - 320057448711568.0,
 -0.688267019144514*x10 - 517863830367896.0,
 -0.495605441561502*x10 - 837921279079464.0,
 -1.30630100789441*x10 - 517863830367896.0,
 -0.311732980855484*x10 + 517863830367898.0,
 1.1136394303114*x10 + 837921279079462.0]

# Isometry

In [56]:
# Will add later

# SDP

## Adjaceny matrix

In [58]:
n = 5
E = np.matrix([[0,1,0,0,1],
               [1,0,1,0,0],
               [0,1,0,1,0],
               [0,0,1,0,1],
               [1,0,0,1,0]])

## Statistics

In [68]:
a = 1/np.sqrt(5)
P_list = [a,0,a,a,a]
P_list[0]

0.44721359549995793

## Optimization

In [70]:
# Create a symmetric matrix variable.
X = cp.Variable((n+1,n+1), symmetric=True)


# TODO: define objective function 
C = np.eye(n+1,n+1)


# Constraints
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [X[0,0] == 1]
for i in range(1,n+1):
    for j in range(1,n+1):
        if E[i-1,j-1] == 1:
            constraints += [X[i,j] == 0]
for i in range(1,n+1):
    constraints += [X[0,i] == X[i,i]]
    constraints += [X[i,i] == P_list[i-1]]
    
prob = cp.Problem(cp.Maximize(cp.trace(C@X)-1),
                  constraints)
prob.solve()

1.7888543852848131